In [3]:
# Prepare data for tokenizer
!python3 tokenizer/prepare-visfd.py \
    ./data/Train.csv \
    --out_dir ./tokenizer

^C
Traceback (most recent call last):
  File "/home/ptdat/Desktop/ViSFD/tokenizer/prepare-visfd.py", line 20, in <module>
    visfd["lang"] = visfd.comment.map(detect)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ptdat/Desktop/venv/lib/python3.12/site-packages/pandas/core/series.py", line 4700, in map
    new_values = self._map_values(arg, na_action=na_action)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ptdat/Desktop/venv/lib/python3.12/site-packages/pandas/core/base.py", line 921, in _map_values
    return algorithms.map_array(arr, mapper, na_action=na_action, convert=convert)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ptdat/Desktop/venv/lib/python3.12/site-packages/pandas/core/algorithms.py", line 1743, in map_array
    return lib.map_infer(values, mapper, convert=convert)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "lib.pyx", line 2972, in pandas._libs.lib.map_infe

In [6]:
# Build tokenizer
!python3 tokenizer/build-tokenizer.py \
    ./tokenizer/feedbacks.txt \
    --out_dir ./tokenizer \
    --config_file ./config.json

[00:00:00] Pre-processing files (1 Mo)    ░░░░░░░░░░░░░░░░░░                  1%

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[00:00:00] Tokenize words                 ██████████████████ 7688     /     7688[00:00:00] Tokenize words                 ██████████████████ 0        /        0
[00:00:00] Count pairs                    ██████████████████ 7688     /     7688
[00:00:00] Compute merges                 ██████████████████ 3640     /     3640


In [1]:
from modeling.dataset import ViSFDDataset
import torch

task = "mtl"

train_set = ViSFDDataset("data/Train.csv", task_type=task)
val_set = ViSFDDataset("data/Dev.csv", task_type=task)
test_set = ViSFDDataset("data/Test.csv", task_type=task)

2024-05-20 09:29:48 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-05-20 09:29:48 INFO  PosTagger:23 - Loading POS Tagging model
2024-05-20 09:29:50 INFO  NerRecognizer:34 - Loading NER model
2024-05-20 09:29:58 INFO  DependencyParser:32 - Loading Dependency Parsing model


100%|██████████| 2224/2224 [00:01<00:00, 1766.00it/s]


In [2]:
from tokenizers import Tokenizer
from modeling.modules import ViSFD_LSTM_CNN
from modeling.losses import ViSFDLoss

tokenizer: Tokenizer = Tokenizer.from_file("./tokenizer/tokenizer.json")
model = ViSFD_LSTM_CNN(
    tokenizer, 
    task_type=task, 
    embed_dim=768,
    dropout=0.5,
    lstm_hidden_size=512,
    lstm_num_layers=2,
    cnn_kernel_size=5,
    cnn_out_channels=16,
    pooling_out_size=64,
    output_dropout=0.5,
    output_hidden_size=64
)
# model = ViSFD_LSTM_CNN_v2(
#     tokenizer, 
#     task_type=task, 
#     embed_dim=768,
#     dropout=0.5,
#     lstm_hidden_size=512,
#     lstm_num_layers=2,
#     cnn_kernel_size=5,
#     cnn_out_channels=16,
#     pooling_out_size=64,
#     output_dropout=0.5,
#     output_hidden_size=64,
#     output_attn_num_heads=8
# )
loss = ViSFDLoss(
    task_type=task, 
    task_loss_args=dict(
        aspect_alpha=1, 
        aspect_gamma=0,
        polarity_weight=1
    )
)
optimizer = torch.optim.AdamW(model.parameters())

from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
ViSFD_LSTM_CNN                           --
├─Embedding: 1-1                         3,072,000
├─LSTM: 1-2                              11,550,720
├─Conv1d: 1-3                            81,936
├─AdaptiveAvgPool1d: 1-4                 --
├─AdaptiveMaxPool1d: 1-5                 --
├─MTL_ViSFDClassifier: 1-6               --
│    └─AspectClassifier: 2-1             --
│    │    └─Sequential: 3-1              131,851
│    └─PolarityClassifier: 2-2           --
│    │    └─ModuleList: 3-2              1,313,310
Total params: 16,149,817
Trainable params: 16,149,817
Non-trainable params: 0

In [3]:
from modeling.utils import train
from modeling.metrics import AspectF1Score, PolarityF1Score

train(
    model=model,
    loss_fn=loss,
    optimizer=optimizer,
    train_set=train_set,
    validation_set=val_set,
    batch_size=64,
    epochs=15,
    metrics={
        "Aspect F1": AspectF1Score(task_type=task, aspect_threshold=0.5),
        "Polarity F1": PolarityF1Score(task_type=task, aspect_threshold=0.5)
    },
    device="cuda"
)

In [ ]:
from modeling.utils import evaluate
from torch.utils.data import DataLoader

evaluate(
    model=model,
    score_fn=AspectF1Score("mtl", aspect_threshold=0.4),
    data=DataLoader(test_set, 64)
)

 Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

 Evaluation: 100%|██████████| 35/35 [00:01<00:00, 23.75it/s]


0.7587565183639526

In [ ]:
from modeling.utils import mtl_decode

torch.save((model, mtl_decode), "model.pt")